# Configuración Inicial
Métodos un utils en común en los notebooks

In [66]:
# Imports
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import to_date, date_add
from pyspark.sql.types import IntegerType, DecimalType
from pyspark.sql.types import StringType
from pyspark.sql import functions as f
from pyspark.sql.functions import col, lit, when, row_number, concat
from pyspark.sql.window import Window

In [2]:
class MySQLConnector:
    def __init__(self, spark: SparkSession, connection_properties: dict, url: str):
        self.spark = spark
        self.properties = connection_properties
        self.url = url

    def get_dataframe(self, sql_query: str):        
        df = self.spark.read.jdbc(
            url=self.url,
            table=sql_query,
            properties=self.properties
        )
        return df
    
    def save_db(self, df, tabla):
        df.write.jdbc(
            url=self.url,
            table=tabla,
            mode='append',
            properties=self.properties
        )
        
def create_spark_session(path_jar_driver):    
    conf = SparkConf().set('spark.driver.extraClassPath', path_jar_driver)
    spark_context = SparkContext(conf=conf)
    sql_context = SQLContext(spark_context)
    return sql_context.sparkSession    

def get_dataframe_from_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

In [32]:
db_user = 'Estudiante_65_202415'
db_psswd = 'Estudiante_202010409'

connection_properties = {
    "user": db_user,
    "password": db_psswd,
    "driver": "com.mysql.cj.jdbc.Driver",
    "batchsize": '500'
}

source_db_string_connection = 'jdbc:mysql://157.253.236.120:8080/RaSaTransaccional_ETL'
destination_db_string_connection = f'jdbc:mysql://157.253.236.120:8080/{db_user}'

# Driver de conexion
# LINUX
path_jar_driver = '/opt/mysql/lib/mysql-connector-java-8.0.28.jar'
# WINDOWS
#path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [33]:
spark = create_spark_session(path_jar_driver)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_3857/885711144.py:25 

In [12]:
conn_orig = MySQLConnector(spark=spark, connection_properties=connection_properties, url=source_db_string_connection)
conn_dest = MySQLConnector(spark=spark, connection_properties=connection_properties, url=destination_db_string_connection)

# Consultar Dimensiones

In [13]:
# Areas de Servicio
sql_dim_areas_servicio = '''(SELECT IdAreaDeServicio_DWH, IdAreaDeServicio_T FROM Rs_AreasDeServicio ORDER BY IdAreaDeServicio_DWH, IdAreaDeServicio_T ASC) AS AreasDeServicio'''
dim_areas_servicio = conn_dest.get_dataframe(sql_dim_areas_servicio)

In [14]:
# Mini condiciones
sql_dim_condiciones = '''(SELECT IdCondicionesBeneficios_DWH AS IdCondicionesTipoBeneficio_DWH, IdTipoBeneficio_T, Annio FROM Rs_MiniCondicionesTipoBeneficio ORDER BY IdCondicionesBeneficios_DWH, IdTipoBeneficio_T, Annio ASC) AS MiniCondicionesTipoBeneficio'''
dim_mini_condiciones = conn_dest.get_dataframe(sql_dim_condiciones)

In [15]:
# Proveedor
sql_dim_proveedores = '''(SELECT IdProveedor_DWH, IdProveedor_T FROM Rs_Proveedor ORDER BY IdProveedor_DWH, IdProveedor_T ASC) AS Proveedor'''
dim_proveedores = conn_dest.get_dataframe(sql_dim_proveedores)

In [16]:
# Niveles de Servicio
sql_dim_nivel_servicio = '''(SELECT IdNivelDeServicio_DWH, IdNivelDeServicio_T FROM Rs_NivelesDeServicio ORDER BY IdNivelDeServicio_DWH, IdNivelDeServicio_T ASC) AS NivelesDeServicio'''
dim_nivel_servicio = conn_dest.get_dataframe(sql_dim_nivel_servicio)

In [17]:
# Tipos Beneficio
sql_dim_tipos_beneficio = '''(SELECT IdTipoBeneficio_DWH, IdTipoBeneficio_T FROM Rs_TiposBeneficio ORDER BY IdTipoBeneficio_DWH, IdTipoBeneficio_T ASC) AS TiposBeneficio'''
dim_tipos_beneficio = conn_dest.get_dataframe(sql_dim_tipos_beneficio)

In [18]:
# Fecha
sql_dim_fecha = '''(SELECT IdFecha AS IdFechaEmision FROM Rs_Fecha ORDER BY IdFecha ASC) AS Rs_Fecha'''
dim_fecha = conn_dest.get_dataframe(sql_dim_fecha)

dim_fecha.show()

+--------------+
|IdFechaEmision|
+--------------+
|      20171231|
|      20181231|
|      20191231|
|      20201231|
|      20211231|
+--------------+



In [19]:
# Condiciones de Pago 
sql_dim_condiciones_coseguro = '''(
SELECT IdCondicionDePago_DWH AS IdCondicionDePagoCoseguro_DWH, IdCondicionDePago_T AS IdCondPagoCoseguro_T
FROM Rs_CondicionDePago
WHERE Tipo = 'Coseguro'
ORDER BY IdCondicionDePago_DWH, IdCondicionDePago_T ASC) AS coseguro'''
dim_coseguro = conn_dest.get_dataframe(sql_dim_condiciones_coseguro)

sql_dim_condiciones_copago = '''(
SELECT IdCondicionDePago_DWH AS IdCondicionDePagoCopago_DWH, IdCondicionDePago_T AS IdCondPagoCopago_T
FROM Rs_CondicionDePago
WHERE Tipo = 'Copago'
ORDER BY IdCondicionDePago_DWH, IdCondicionDePago_T ASC
) AS copago'''
dim_copago = conn_dest.get_dataframe(sql_dim_condiciones_copago)

Se muestra la estructura cargada

In [20]:

dim_tipos_beneficio.printSchema()
dim_mini_condiciones.printSchema()
dim_areas_servicio.printSchema()
dim_nivel_servicio.printSchema()
dim_coseguro.printSchema()
dim_copago.printSchema()
dim_proveedores.printSchema()
dim_fecha.printSchema()


root
 |-- IdTipoBeneficio_DWH: integer (nullable = true)
 |-- IdTipoBeneficio_T: integer (nullable = true)

root
 |-- IdCondicionesTipoBeneficio_DWH: integer (nullable = true)
 |-- IdTipoBeneficio_T: integer (nullable = true)
 |-- Annio: integer (nullable = true)

root
 |-- IdAreaDeServicio_DWH: integer (nullable = true)
 |-- IdAreaDeServicio_T: integer (nullable = true)

root
 |-- IdNivelDeServicio_DWH: integer (nullable = true)
 |-- IdNivelDeServicio_T: integer (nullable = true)

root
 |-- IdCondicionDePagoCoseguro_DWH: integer (nullable = true)
 |-- IdCondPagoCoseguro_T: integer (nullable = true)

root
 |-- IdCondicionDePagoCopago_DWH: integer (nullable = true)
 |-- IdCondPagoCopago_T: integer (nullable = true)

root
 |-- IdProveedor_DWH: integer (nullable = true)
 |-- IdProveedor_T: integer (nullable = true)

root
 |-- IdFechaEmision: integer (nullable = true)



# Hecho Planes tipo Beneficio

## Extraction

Se consulto la tabla fuente haciendo uso de cast y when de modificaron los campos para obtener la informacion requirida de manera limpia, y finalmente se eliminaron duplicados.

In [21]:
# Your SQL query remains the same
sql_fuente_planes = '''(
SELECT DISTINCT
    IdTipoBeneficio_T AS IdTipoBeneficio, 
    IdAreaDeServicio_T AS IdAreaDeServicio,
    IdCondicionDePagoCoseguro_T AS IdCondicionDePagoCoseguro,
    IdCondicionDePagoCopago_T AS IdCondicionDePagoCopago,
    IdNivelServicio_T AS IdNivelDeServicio,
    IdPlan_T AS IdPlan_DD,
    IdProveedor_T AS IdProveedor,    
    CASE
        WHEN Fecha REGEXP '^[0-9]{4}-[0-9]{2}-[0-9]{2}$'
            THEN CAST(REPLACE(Fecha, '-', '') AS UNSIGNED)
        WHEN Fecha REGEXP '^[A-Za-z]{3} [0-9]{1,2},[0-9]{4}$'
            THEN CAST(DATE_FORMAT(STR_TO_DATE(TRIM(Fecha), '%b %d,%Y'), '%Y%m%d') AS UNSIGNED)        
    END AS FechaID,
    COALESCE(valorCoseguro, 0) AS valorCoseguro,
    COALESCE(valorCopago, 0) AS valorCopago,
    COALESCE(cantidadLimite, 0) AS cantidadLimite
FROM
    FuentePlanesBeneficio_ETL
ORDER BY
    IdTipoBeneficio,
    IdAreaDeServicio,
    IdCondicionDePagoCoseguro,
    IdCondicionDePagoCopago,
    IdNivelDeServicio,
    IdPlan_DD,
    IdProveedor,
    FechaID
) AS FuentePlanesBeneficio_ETL'''

df_fuente_planes = conn_orig.get_dataframe(sql_fuente_planes)

df_fuente_planes = df_fuente_planes \
    .withColumn('IdTipoBeneficio', col('IdTipoBeneficio').cast(IntegerType())) \
    .withColumn('IdAreaDeServicio', col('IdAreaDeServicio').cast(IntegerType())) \
    .withColumn('IdCondicionDePagoCoseguro', col('IdCondicionDePagoCoseguro').cast(IntegerType())) \
    .withColumn('IdCondicionDePagoCopago', col('IdCondicionDePagoCopago').cast(IntegerType())) \
    .withColumn('IdNivelDeServicio', col('IdNivelDeServicio').cast(IntegerType())) \
    .withColumn('IdPlan_DD', col('IdPlan_DD').cast(StringType())) \
    .withColumn('IdProveedor', col('IdProveedor').cast(IntegerType())) \
    .withColumn('FechaID', col('FechaID').cast(IntegerType())) \
    .withColumn('valorCoseguro', col('valorCoseguro').cast(DecimalType(18, 2))) \
    .withColumn('valorCopago', col('valorCopago').cast(DecimalType(18, 2)))  \
    .withColumn('cantidadLimite', col('cantidadLimite').cast(IntegerType())) 
    

In [22]:
df_fuente_planes.show()

+---------------+----------------+-------------------------+-----------------------+-----------------+-----------------+-----------+--------+-------------+-----------+--------------+
|IdTipoBeneficio|IdAreaDeServicio|IdCondicionDePagoCoseguro|IdCondicionDePagoCopago|IdNivelDeServicio|        IdPlan_DD|IdProveedor| FechaID|valorCoseguro|valorCopago|cantidadLimite|
+---------------+----------------+-------------------------+-----------------------+-----------------+-----------------+-----------+--------+-------------+-----------+--------------+
|              5|          312017|                       45|                    238|                1|11269WY0080014-00|      11269|20171231|         0.00|       0.00|             0|
|              5|          312017|                       45|                    238|                2|11269WY0080014-00|      11269|20171231|         0.00|       0.00|             0|
|              5|          312017|                       45|                    238| 

## Transformation

Tabla temporal para unir Tipos de beneficio y mini condiciones de tipo de beneficio

In [23]:
df_condiciones_tipo_beneficio = (
    dim_mini_condiciones.alias('mini')
    .join(
        dim_tipos_beneficio.alias('tb'),
        col('mini.IdTipoBeneficio_T') == col('tb.IdTipoBeneficio_T'),
        'left'
    )
    .select(
        col('mini.IdCondicionesTipoBeneficio_DWH'),
        col('mini.IdTipoBeneficio_T'),
        col('mini.Annio'),
        col('tb.IdTipoBeneficio_DWH')
    )
)

df_condiciones_tipo_beneficio.show()

+------------------------------+-----------------+-----+-------------------+
|IdCondicionesTipoBeneficio_DWH|IdTipoBeneficio_T|Annio|IdTipoBeneficio_DWH|
+------------------------------+-----------------+-----+-------------------+
|                             0|                0|    0|                  0|
|                             1|                5| 2017|                  1|
|                             2|                5| 2020|                  1|
|                             3|               10| 2017|                  2|
|                             4|               10| 2019|                  2|
|                             5|               20| 2019|                  3|
|                             6|               25| 2018|                  4|
|                             7|               30| 2020|                  5|
|                             8|               40| 2017|                  6|
|                             9|               45| 2018|                  7|

Revision de posibles nulos.

In [24]:
columns_to_check = ["IdCondicionesTipoBeneficio_DWH", "IdTipoBeneficio_T", "Annio", "IdTipoBeneficio_DWH", "IdTipoBeneficio_T"]

for col_name in columns_to_check:
    null_count = df_condiciones_tipo_beneficio.filter(col(col_name).isNull()).count()
    if null_count > 0:
        print(f"Se encontraron {null_count} valores nulos en la columna {col_name}.")
    else:
        print(f"No se encontraron valores nulos en la columna {col_name}.")


No se encontraron valores nulos en la columna IdCondicionesTipoBeneficio_DWH.
No se encontraron valores nulos en la columna IdTipoBeneficio_T.
No se encontraron valores nulos en la columna Annio.
No se encontraron valores nulos en la columna IdTipoBeneficio_DWH.
No se encontraron valores nulos en la columna IdTipoBeneficio_T.


In [25]:
from pyspark.sql.functions import col

fact_fuente_planes = (df_fuente_planes.alias('fp')                    
                    .join(df_condiciones_tipo_beneficio.alias('cd'), df_fuente_planes.IdTipoBeneficio == df_condiciones_tipo_beneficio.IdTipoBeneficio_T, 'left')
                    .join(dim_areas_servicio.alias('ase'), df_fuente_planes.IdAreaDeServicio == dim_areas_servicio.IdAreaDeServicio_T, 'left')                    
                    .join(dim_nivel_servicio.alias('ns'), df_fuente_planes.IdNivelDeServicio == dim_nivel_servicio.IdNivelDeServicio_T, 'left')
                    .join(dim_coseguro.alias('cos'), df_fuente_planes.IdCondicionDePagoCoseguro == dim_coseguro.IdCondPagoCoseguro_T, 'left')
                    .join(dim_copago.alias('cop'), df_fuente_planes.IdCondicionDePagoCoseguro == dim_copago.IdCondPagoCopago_T, 'left')
                    .join(dim_proveedores.alias('pro'), df_fuente_planes.IdProveedor == dim_proveedores.IdProveedor_T, 'left')
                    .join(dim_fecha.alias('fec'), df_fuente_planes.FechaID == dim_fecha.IdFechaEmision, 'left')
                    .select([col('cd.IdCondicionesTipoBeneficio_DWH'), col('cd.IdTipoBeneficio_DWH'),
                             col('ase.IdAreaDeServicio_DWH'),
                             col('ns.IdNivelDeServicio_DWH'),
                             col('cos.IdCondicionDePagoCoseguro_DWH'),
                             col('cop.IdCondicionDePagoCopago_DWH'),
                             col('pro.IdProveedor_DWH'),
                             col('fec.IdFechaEmision'),
                             col('fp.IdPlan_DD'),
                             col('fp.ValorCoseguro'), col('fp.ValorCopago'), col('fp.CantidadLimite')
                             ])
                    .fillna({'IdCondicionesTipoBeneficio_DWH': 0, 'IdTipoBeneficio_DWH': 0, 'IdAreaDeServicio_DWH': 0, 'IdNivelDeServicio_DWH': 0, 'IdCondicionDePagoCoseguro_DWH': 0, 'IdCondicionDePagoCopago_DWH': 0, 'IdProveedor_DWH': 0})) 
                    
fact_fuente_planes.show(5)


+------------------------------+-------------------+--------------------+---------------------+-----------------------------+---------------------------+---------------+--------------+-----------------+-------------+-----------+--------------+
|IdCondicionesTipoBeneficio_DWH|IdTipoBeneficio_DWH|IdAreaDeServicio_DWH|IdNivelDeServicio_DWH|IdCondicionDePagoCoseguro_DWH|IdCondicionDePagoCopago_DWH|IdProveedor_DWH|IdFechaEmision|        IdPlan_DD|ValorCoseguro|ValorCopago|CantidadLimite|
+------------------------------+-------------------+--------------------+---------------------+-----------------------------+---------------------------+---------------+--------------+-----------------+-------------+-----------+--------------+
|                             1|                  1|                  31|                    1|                            7|                          0|              0|      20171231|11269WY0080014-00|         0.00|       0.00|             0|
|                       

Revision de duplicados

In [26]:
fact_fuente_planes.count()

184121

In [38]:
columns_to_check = ["IdCondicionesTipoBeneficio_DWH", "IdTipoBeneficio_DWH", "IdAreaDeServicio_DWH", "IdCondicionDePagoCoseguro_DWH", "IdCondicionDePagoCopago_DWH", "IdCondicionDePagoCopago_DWH", "IdProveedor_DWH", "IdFechaEmision",  "IdPlan_DD"]

for col_name in columns_to_check:
    null_count = fact_fuente_planes.filter(col(col_name).isNull()).count()
    if null_count > 0:
        print(f"Se encontraron {null_count} valores nulos en la columna {col_name}.")
    else:
        print(f"No se encontraron valores nulos en la columna {col_name}.")


No se encontraron valores nulos en la columna IdCondicionesTipoBeneficio_DWH.
No se encontraron valores nulos en la columna IdTipoBeneficio_DWH.
No se encontraron valores nulos en la columna IdAreaDeServicio_DWH.
No se encontraron valores nulos en la columna IdCondicionDePagoCoseguro_DWH.
No se encontraron valores nulos en la columna IdCondicionDePagoCopago_DWH.
No se encontraron valores nulos en la columna IdCondicionDePagoCopago_DWH.
No se encontraron valores nulos en la columna IdProveedor_DWH.


No se encontraron valores nulos en la columna IdFechaEmision.
No se encontraron valores nulos en la columna IdPlan_DD.


In [28]:
fact_fuente_planes.printSchema()

root
 |-- IdCondicionesTipoBeneficio_DWH: integer (nullable = false)
 |-- IdTipoBeneficio_DWH: integer (nullable = false)
 |-- IdAreaDeServicio_DWH: integer (nullable = false)
 |-- IdNivelDeServicio_DWH: integer (nullable = false)
 |-- IdCondicionDePagoCoseguro_DWH: integer (nullable = false)
 |-- IdCondicionDePagoCopago_DWH: integer (nullable = false)
 |-- IdProveedor_DWH: integer (nullable = false)
 |-- IdFechaEmision: integer (nullable = true)
 |-- IdPlan_DD: string (nullable = true)
 |-- ValorCoseguro: decimal(18,2) (nullable = true)
 |-- ValorCopago: decimal(18,2) (nullable = true)
 |-- CantidadLimite: integer (nullable = true)



In [29]:
fact_fuente_planes.count()

184121

In [39]:
#columns_to_check = ["IdCondicionesTipoBeneficio_DWH", "IdTipoBeneficio_DWH", "IdAreaDeServicio_DWH", "IdCondicionDePagoCoseguro_DWH", "IdCondicionDePagoCopago_DWH", "IdCondicionDePagoCopago_DWH", "IdProveedor_DWH", "IdFechaEmision"]
fact_fuente_planes_without_duplicates = fact_fuente_planes.dropDuplicates(columns_to_check)
fact_fuente_planes_without_duplicates.count()

107359

## LOAD

In [40]:
conn_dest.save_db(fact_fuente_planes_without_duplicates, "Rs_HechoPlanesTiposBeneficio")

# Hecho Historia Condiciones Tipo Beneficio

## Extraction  
Se reusa la tabla temporal de mini condiciones tipo beneficio de la parte de arriba



In [42]:
df_condiciones_tipo_beneficio.show()

+------------------------------+-----------------+-----+-------------------+
|IdCondicionesTipoBeneficio_DWH|IdTipoBeneficio_T|Annio|IdTipoBeneficio_DWH|
+------------------------------+-----------------+-----+-------------------+
|                             0|                0|    0|                  0|
|                             1|                5| 2017|                  1|
|                             2|                5| 2020|                  1|
|                             3|               10| 2017|                  2|
|                             4|               10| 2019|                  2|
|                             5|               20| 2019|                  3|
|                             6|               25| 2018|                  4|
|                             7|               30| 2020|                  5|
|                             8|               40| 2017|                  6|
|                             9|               45| 2018|                  7|

## Extraction

In [68]:
df_condiciones_tipo_beneficio_temp = df_condiciones_tipo_beneficio.withColumn("IdFechaInicio", concat(col("Annio").cast("string"), lit("1231")))

window_spec = Window.partitionBy("IdTipoBeneficio_T").orderBy(col("Annio").desc())

df_condiciones_tipo_beneficio_temp = df_condiciones_tipo_beneficio_temp.withColumn("Cambio", when(row_number().over(window_spec) == 1, "Yes").otherwise("No"))

df_condiciones_tipo_beneficio_temp = df_condiciones_tipo_beneficio_temp.withColumn(
    "end_date",
    when(col("Cambio") == "Yes", to_date(lit("99991231"), "yyyyMMdd"))  
    .otherwise(
         date_add(to_date(concat(col("Annio").cast("string"), lit("1231")), "yyyyMMdd"), 365)
    )
)

df_condiciones_tipo_beneficio_temp = df_condiciones_tipo_beneficio_temp.withColumn("IdFechaFin", f.date_format("end_date", "yyyyMMdd").cast("int"))
df_condiciones_tipo_beneficio_temp.show()

+------------------------------+-----------------+-----+-------------------+-------------+------+----------+----------+
|IdCondicionesTipoBeneficio_DWH|IdTipoBeneficio_T|Annio|IdTipoBeneficio_DWH|IdFechaInicio|Cambio|  end_date|IdFechaFin|
+------------------------------+-----------------+-----+-------------------+-------------+------+----------+----------+
|                             0|                0|    0|                  0|        01231|   Yes|9999-12-31|  99991231|
|                             2|                5| 2020|                  1|     20201231|   Yes|9999-12-31|  99991231|
|                           331|                5| 2018|                  1|     20181231|    No|2019-12-31|  20191231|
|                             1|                5| 2017|                  1|     20171231|    No|2018-12-31|  20181231|
|                           273|               10| 2021|                  2|     20211231|   Yes|9999-12-31|  99991231|
|                             4|        

In [88]:
fact_PlanesTiposBeneficio = df_condiciones_tipo_beneficio_temp.select(col("IdCondicionesTipoBeneficio_DWH"), col("IdTipoBeneficio_DWH"), col("IdFechaInicio"), col("IdFechaFin"), col("Cambio"))


# LOAD

In [90]:
columns_to_check = ["IdCondicionesTipoBeneficio_DWH", "IdTipoBeneficio_DWH", "IdFechaInicio", "IdFechaFin", "Cambio"]

for col_name in columns_to_check:
    null_count = fact_PlanesTiposBeneficio.filter(col(col_name).isNull()).count()
    if null_count > 0:
        print(f"Se encontraron {null_count} valores nulos en la columna {col_name}.")
    else:
        print(f"No se encontraron valores nulos en la columna {col_name}.")

No se encontraron valores nulos en la columna IdCondicionesTipoBeneficio_DWH.
No se encontraron valores nulos en la columna IdTipoBeneficio_DWH.
No se encontraron valores nulos en la columna IdFechaInicio.
No se encontraron valores nulos en la columna IdFechaFin.
No se encontraron valores nulos en la columna Cambio.


In [92]:
fact_PlanesTiposBeneficio = fact_PlanesTiposBeneficio.withColumn('IdFechaInicio', col('IdFechaInicio').cast(IntegerType()))

In [93]:
fact_PlanesTiposBeneficio.printSchema()

root
 |-- IdCondicionesTipoBeneficio_DWH: integer (nullable = true)
 |-- IdTipoBeneficio_DWH: integer (nullable = true)
 |-- IdFechaInicio: integer (nullable = true)
 |-- IdFechaFin: integer (nullable = true)
 |-- Cambio: string (nullable = false)



In [97]:
fact_PlanesTiposBeneficio.select(col("IdFechaInicio")).distinct().show()

+-------------+
|IdFechaInicio|
+-------------+
|     20211231|
|         1231|
|     20191231|
|     20171231|
|     20181231|
|     20201231|
+-------------+



In [99]:
fact_PlanesTiposBeneficio = fact_PlanesTiposBeneficio.filter(col("IdCondicionesTipoBeneficio_DWH") != 0)
fact_PlanesTiposBeneficio.show()

+------------------------------+-------------------+-------------+----------+------+
|IdCondicionesTipoBeneficio_DWH|IdTipoBeneficio_DWH|IdFechaInicio|IdFechaFin|Cambio|
+------------------------------+-------------------+-------------+----------+------+
|                             2|                  1|     20201231|  99991231|   Yes|
|                           331|                  1|     20181231|  20191231|    No|
|                             1|                  1|     20171231|  20181231|    No|
|                           273|                  2|     20211231|  99991231|   Yes|
|                             4|                  2|     20191231|  20201230|    No|
|                           325|                  2|     20181231|  20191231|    No|
|                             3|                  2|     20171231|  20181231|    No|
|                           289|                178|     20201231|  99991231|   Yes|
|                           318|                178|     20191231

In [108]:
fact_PlanesTiposBeneficio = fact_PlanesTiposBeneficio.select(col("IdTipoBeneficio_DWH"), col("IdCondicionesTipoBeneficio_DWH"), col("IdFechaInicio"), col("IdFechaFin"), col("Cambio"))

In [115]:
conn_dest.save_db(fact_PlanesTiposBeneficio, "Rs_HechoHistCondicionesTiposBeneficio")